# CNN Architectures for large-scale audio classification(2)

Abstract

---

1. Introduction

CNNは音分類にも有用なのか試したい。
我々は7000万のトレーニングビデオと30,871のラベルを持つYoutube-100Mというデータセットを用意した。
音情報を用いてビデオ単位でのラベル付けを優先タスクとする。
以下のことを調査する。
有名なDNNアーキテクチャはビデオの音分類のタスクでどのように比較されるか。
トレーニングセットとラベルセットの大きさを変更して、それが精度にどのような違いをもたらすか。
今回は時間的に何も変化させていないので、RNNは用いない。

視覚ベースのビデオ分類[20]をまねて、単一の分類器に、ビデオを重複の無いセグメントに分割したものを入力し、それぞれのセグメントの予測結果を平均するという方法を採る。[20]の研究者らは、時間を超えた情報を結び付けるためのより複雑なモデルの調査をしたのち、単一フレームを入力とするCNN分類器の出力を平均化したものが複雑なモデルと似た性能を示すことを発見した。
[21]内で、AEDタスクはマルチインスタンス学習問題の1つとみなせると考えたが、マルチインスタンス学習のスケーリング問題はまだ解決できていないとも述べている。対照的に我々は、より巨大なデータセットに対して弱ラベルを用いた学習の限界について調査した。多くの個々のセグメントが情報を持たないのに対して、我々は十分な学習が行われれば、NETは有用な手掛かりに目をつける方法を学ぶことを望んでいる。我々は、どれくらいラベルが「弱い」かを定量化することはできない(つまり、どれくらいの割合のセグメントが情報を持たないか)し、大部分のクラス(例えばコンピューターハードウェア、ボーイング757、ビー玉(オーリー))に対して、関係性を決定付ける明確な方法ではない。いくつかのクラス(ビーチとか)については、背景の雰囲気はそれ自体情報を持つ。

我々はデータセットのサイズによって、巨大なモデル容量のあるネットワークを調べることができ、完全に画像分類のアイディアを活用できる。複数のフレームのログメルスペクトグラムを計算することによって我々は分類器に入力する、二次元画像のようなパッチを作製する。時間軸と頻度軸の明らかに異なる意味が音特有のアーキテクチャに合わないかもしれないが、この試みは、Inception-V3やResNet-50といった最小限に変えた画像分類ネットワークを用いている。我々はトレーニングセットのサイズが性能に与える影響を評価するために、23K～70MのYoutube-100Mのサブセットで学習させ、サブセットのラベルを400~300Kの間で変えてモデルを学習させることで、一般的にラベルセットのサイズがどれくらい影響するかを調査する。ここで、性能は単一の共通するラベルセットを用いて評価する。我々は、AudioSetデータセット上で、我々のネットワークの一つからUmbeddingで学習させたモデルの性能を調べることで、AEDに我々のネットワークがどれだけ有効なのかを調査する。

---

2. Dataset

Youtube-100Mは、1億のYoutubeビデオから構成されており、7000万ビデオが学習用、1000万ビデオが評価用、そして残りの2000万ビデオがバリデーション用のものである。ビデオは平均4.6分で、合計540万時間(学習)にもなる。これらのビデオのそれぞれは、(knowledge graph:知識グラフから)30,871個ラベルから一つ以上の識別子でラベル付けされている。一つのラベルには平均して約5ラベルが付けられている。ラベルはそれぞれのビデオのメタデータ(タイトル、説明、コメント)や、背景、画像の内容の組み合わせに基づいて、自動的に割り当てられている。ラベルはビデオ全体に適用され、非常に包括的なもの(例えば、歌)から特定のもの(ウ(水鳥))まである。表1はラベルの例である。
　機械が生成したので、ラベルは100％正確ではないし、3000のラベルのうちいくつかは明らかに音響的に関係がある(トランペットなど)一方で、かなりそうではないもの(Web Pageなど)もある。動画には複数の具体的なアノテーションが行われている。例えば、トランペットとラベリングされたビデオは、なんら階層性がないのに、よく「エンターテインメント」としてもラベル付けされている。

---

3. Experimental framework

3.1 Traning  
 音は重複なく、960[ms]の長さのフレームに分けられる。これは学習用の7000万本(524万時間)の動画からおよそ200億の例を取得できる。それぞれのフレームは動画全体のラベルを全て引き継ぐ。960[ms]のフレームは、短時間フーリエ変換(STFT)を10[ms]ごとに25[ms]の窓関数に適用して分解される。結果的に生じたスペクトログラムは、メル間隔度数のビンにまとめられ、そして数的な問題を避けるために、小さなオフセットを追加した後にそれぞれのビンの大きさは対数変換される。これは、すべての分類器に対する入力を形作る96x64のビンのログメルスペクトログラムのパッチを与える。トレーニングの間、我々は全てのパッチからランダムにサンプリングすることで、128個の入力からなるミニバッチを用意した。
 　全ての実験はTensorFlowを用い、そして複数のGPU上でAdam[23]最適化を用いて非同期で学習が行われた。
我々はgrid searchを、Learning rate, batch size、GPUの数、parameter serverに対して行った(ハイパーパラメータの最適化)。Batch Normalizationは後に全ての畳み込み層に適用された。すべてのモデルで、最終層にはソフトマックスではなくシグモイドが用いられた(それぞれの例が複数ラベルを持つため)。損失関数は交差エントロピー誤差。学習セットの大きさを考慮して、dropout, weight decay, その他の一般的な正則化技術は用いていない。700万以上の例で学習させた場合には過学習の傾向は全く見られなかった。学習中、我々は検証セットを用いて最高精度と[mAP](https://kikaben.com/mean-average-precision/)(mean Average Precision)で途中経過を監視した。 

3.2 Evaluation  
　1000万の評価ビデオから、我々はバランスのとれた3つのセット(おおよそ33の例が各クラスに含まれている)を作った。つまり、3万個のラベルに対して100万本の動画が対応し、3087(以降3000)個の最も高い頻度のラベルに対しては10万本のビデオが対応し、400個の最も高い頻度のラベルに対しては1万2000本の動画が対応する。それぞれの評価ビデオから得られた各960[ms]のフレームを分類器に渡した。そして、動画内の全てのセグメントから得られた分類器の出力スコアを平均化した。
 評価基準に対して、我々は、AUC(等価なd-primeクラス分離としても報告されている)や、mAP(mean Average Precision)の全てのクラスに渡って、バランスの取れた平均を計算した。AUC(Area under the curve)は、ROC(受信者動作特性)曲線(Receiver Operationg Characteristic curve)の下の面積のことで、つまり誤ってNegativeであるものをPositiveであると分類した確率(偽陽性)の関数として正しく陽性だと分類した確率(正答率)のことである。そして完全な分類のAUCは1.0(感度d'=∞の場合と一致する)になり、あてずっぽう(d'=0)¹だとAUCは0.5になる。ｍAPはクラス群のAP(Average Precision)の平均のことで、APはそれぞれ個々のPositive試行を含むくらい十分に長い表に渡って、平均化された試行ランキング表におけるPositiveの割合(つまり適合率のことである。APは特定の回復リスト長を必要としない、適合率の指標として広く用いられてきたが、AUCとは違って、クラスの事前確率と直接的に相互関係を持つ。我々のクラスのほとんどが極めて低い事前確率(<10^-4)であるため、空振り率(False alarm rate)は低いとしても、我々が報告したｍAPは典型的に小さい。
 [1]d' = √2 * F^-1(AUC)　ここで、F^-1は単位ガウス分布の逆累積分布関数である。

3.3 Architecture  
　我々の基準は完全結合DNNで、そしてそれは成功している画像分類器を詳細に模倣したいくつかのネットワークと比較された。我々の基準となる実験では、学習と評価を、元の3万ラベルのうち最も頻度の高い10％のラベル(つまり3000ラベル)のみを用いて行った。それぞれの実験で、我々はフレーム単位での分類精度のために、GPUの数とLearning Rateを粗く最適化した。最適なGPUの数は全体の演算能力と通信オーバーヘッドの間での妥協を表しており、それはアーキテクチャによって異なる。

3.3.1 Fully Connected
 我々の基準となるネットワークはReLU活性化、N層、各層につきMニューロンの完全結合モデルである。NとMは以下の、N = [2, 3, 4, 5, 6], M = [500, 1000, 2000, 3000, 4000]の中から探した。結果として、最も性能の良いモデルはN = 3, M = 1000, Learning rate = 3x10^-5の時で、10個のGPUと5つのパラメータサーバを用いた。このネットワークはおおよそ1120万個の重みと1120万個の乗数を持つ。

3.3.2 AlexNet  
　オリジナルのAlexNetアーキテクチャは224x224x3の入力サイズと、stride = 4の頭の11x11畳み込み層を持つ。我々の入力サイズは96x64であるため、活性化層が入力層の後の層で似るように、stride = 2 x 1として用いた。また、Local response normalization(LRN)の代わりにそれぞれの畳み込み層後にbatch normalizationを用い、1000ニューロンの最終層を3087個のニューロンに置き換えた。オリジナルのAlexNetがおおよそ6240万個の重みと11億個の乗数を持つが、我々のバージョンは3730万個の重みと7億6700万個の乗数を持つ。さらに、簡単のために、AlexNetとは違い、複数のデバイス間でフィルターを分割しない。そして、20個のGPUと10個のパラメータサーバーを用いて学習させた。

3.3.3 VGG  
　我々がVGG(Configuration: E)に対して行ったわずかな変更は、LRN(Local response normalization)の代わりにbatch normalizationを用いたことと、活性化関数としてsigmoidを用いた3087ニューロン最終層に対するものである。オリジナルのネットワークが1億4400万個の重みと200億の乗数を持つが、音を用いるバージョンは6200万個の重みと24億の乗数を持つ。(AlexNetに対して行ったように)始めのstrideを減らした他のバージョンも試したが、strideを修正しない方が早く学習でき、よい性能を示すことが分かった。我々の設定で、10個より多いGPUで並列処理することは、大きく役立つわけではなかったので、10個のGPUと5つのパラメータサーバで学習を行った。

表2: 3000個のラベル付けがそれぞれ為された7000万本の動画を用いて学習させた複数のDNNアーキテクチャのパフォーマンスの比較
　最終行は他のモデルよりもLearning rateを1300万ステップ後に減少させることで、かなり長く学習させたモデルの結果を含んでいる。

3.3.4 Inception V3
 我々はinception V3のネットワークの補助ネットワークを取り除くだけでなく、MaxPool以下であり、幹である最初の4層を取り除くことで修正した。そして活性化の変化を反映させるためにAveragePoolサイズを10x6に変更した。我々は幹を残してみたり、最初のstride = 2とMaxPoolを取り除いたり試したが、幹を取り除いたバージョンよりも性能が悪くなることが分かった。オリジナルのネットワークは2700万の重みと56億の乗数を持っており、音に対応したバージョンでは2800万の重みと47億の乗数を持つ。我々は40個のGPUと20個のパラメータサーバを用いた。

3.3.5 ResNet-50
　我々はResNet-50を、活性化の数が音に対応したバージョンにおいて大きく異ならないように、最初の7x7の畳み込みにおけるstride = 2を取り除くことで修正した。そしてAveragePoolサイズを、活性化の変更に対応させるため、6x4に変更した。オリジナルのネットワークが2600万個の重みと38億の乗数を持つのに対して、音に対応させたバージョンは3000万個の重みと19億の乗数を持つ。学習は20個のGPUと10個のパラメータサーバーを用いて行った。

---

4. Experiment